# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [66]:
import boto3
import json

In [67]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [68]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [69]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

{'ResponseMetadata': {'RequestId': 'b876151d-7c14-45c3-a995-4adc083b4530',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 11 Sep 2020 11:44:23 GMT',
   'x-amzn-requestid': 'b876151d-7c14-45c3-a995-4adc083b4530',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [74]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteEventTracker operation: Resource Arn arn:aws:personalize:us-east-1:051545784337:event-tracker/5ec93b79 does not exist.

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

Note: You will not be able to delete the filters until the campaign deletion completes, which may take a few minutes

In [78]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

ResourceInUseException: An error occurred (ResourceInUseException) when calling the DeleteFilter operation: The dataset group the filter belongs to is referenced by a campaign that's being actively created. Please wait for the resource creation to complete, and try the DeleteFilter call again

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [57]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteSolution operation: The given solution does not exist: arn:aws:personalize:us-east-1:051545784337:solution/personalize-poc-userpersonalization

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [58]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteDataset operation: Resource Arn arn:aws:personalize:us-east-1:051545784337:dataset/personalize-poc-movielens25m/ITEMS does not exist.

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [59]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': 'fca8f06c-065d-4ec0-9806-fcd889854fbe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 23:47:24 GMT',
   'x-amzn-requestid': 'fca8f06c-065d-4ec0-9806-fcd889854fbe',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [60]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

{'ResponseMetadata': {'RequestId': 'f1eaa0f0-86a8-41f6-b9c9-96beb96c080f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 23:47:27 GMT',
   'x-amzn-requestid': 'f1eaa0f0-86a8-41f6-b9c9-96beb96c080f',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [61]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [62]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'},
  {'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '600c3316-0318-4f8f-82b4-cf42847aa88d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '600c3316-0318-4f8f-82b4-cf42847aa88d',
   'content-type': 'text/xml',
   'content-length': '728',
   'date': 'Thu, 10 Sep 2020 23:47:31 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [63]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': '7820c201-4749-46d8-99f5-b1558d9e183d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7820c201-4749-46d8-99f5-b1558d9e183d',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 10 Sep 2020 23:47:33 GMT'},
  'RetryAttempts': 0}}

In [64]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': '1a6c1a7b-e559-49ce-8f45-8f8abacb7b47',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1a6c1a7b-e559-49ce-8f45-8f8abacb7b47',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 10 Sep 2020 23:47:35 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [65]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': '89f9978b-8834-4a55-acac-8ded1bb31ea9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '89f9978b-8834-4a55-acac-8ded1bb31ea9',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 10 Sep 2020 23:47:37 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.